In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime

In [2]:
# Start with a date range - Last 30 days 
start = datetime.datetime.today() - datetime.timedelta(days=30)
end   = datetime.datetime.today()

tickers = ["DGS1MO", "DGS3MO", "DGS6MO", "DGS1", "DGS2", "DGS5", "DGS10", "DGS20", "DGS30"]
df = web.DataReader(tickers, "fred", start, end)

In [3]:
# Drop missing rows and get the most recent dates row 
df = df.dropna()
latest = df.iloc[-1]

# Define maturities in years for the x - coords 
maturities = np.array([1/12, 0.25, 0.5, 1, 2, 5, 10, 20, 30])  # years

# Get yields in % for the y - coords 
yields = latest.values

# Stack into an (n x 2) NumPy array
Y = np.column_stack([maturities, yields])
print(Y)

[[ 0.08333333  4.2       ]
 [ 0.25        4.02      ]
 [ 0.5         3.83      ]
 [ 1.          3.68      ]
 [ 2.          3.6       ]
 [ 5.          3.74      ]
 [10.          4.16      ]
 [20.          4.71      ]
 [30.          4.73      ]]


In [7]:
def coeff(Y):
    """
    Parameters
    ----------
    Y : (n,2) numpy array
        Column 0 = x (maturities)
        Column 1 = y (yields)

    Returns
    -------
    coeffs : (2n-2, 4) numpy array
    uses natural boundary conditions
    """
    
    x = Y[:, 0]   # maturities
    y = Y[:, 1]   # yields
    n = x.shape[0]

    h = np.diff(x)

    # ---------------------------
    # Build A
    # ---------------------------
    main_diag = 2 * (h[:-1] + h[1:])
    off_diag = h[1:-1]
    A = np.diag(main_diag) + np.diag(off_diag, 1) + np.diag(off_diag, -1)

    # ---------------------------
    # Build z
    # ---------------------------
    diff_x = h
    diff_y = np.diff(y)

    z_x = 6 * (diff_x[1:] / h[1:] - diff_x[:-1] / h[:-1])
    z_y = 6 * (diff_y[1:] / h[1:] - diff_y[:-1] / h[:-1])

    z = np.column_stack([z_x, z_y])

    # ---------------------------
    # Solve Ac = z
    # ---------------------------
    c = np.linalg.solve(A,z)
    c_x = np.zeros(n); c_y = np.zeros(n)
    c_x[1:-1] = c[:, 0]
    c_y[1:-1] = c[:, 1]

    # ---------------------------
    # Find Coefficients
    # ---------------------------
    a_x = x[:-1]
    b_x = (diff_x / h) - (h/6) * (2*c_x[:-1] + c_x[1:])
    c_x_coef = c_x[:-1] / 2
    d_x = (c_x[1:] - c_x[:-1]) / (6*h)

    a_y = y[:-1]
    b_y = (diff_y / h) - (h/6) * (2*c_y[:-1] + c_y[1:])
    c_y_coef = c_y[:-1] / 2.0
    d_y = (c_y[1:] - c_y[:-1]) / (6*h)

    coeff_x = np.column_stack([a_x, b_x, c_x_coef, d_x])
    coeff_y = np.column_stack([a_y, b_y, c_y_coef, d_y])

    return np.vstack([coeff_x, coeff_y])